In [3]:
import cv2
from PIL import Image
import numpy as np

def dct_compress_image(input_path, output_path, quality=50):

    image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Error: Unable to read image from {input_path}")
        return

    height, width = image.shape

    # Pad image to make it a multiple of 8
    pad_h = (8 - (height % 8)) % 8
    pad_w = (8 - (width % 8)) % 8
    image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)

    new_height, new_width = image.shape

    # JPEG-like quantization matrix
    Q = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])

    # quantization matrix based on quality factor
    if quality < 50:
        scale = 50 / quality
    else:
        scale = 2 - (quality / 50)

    Q = np.clip(Q * scale, 1, 255).astype(np.uint8)  # Prevent zero division


    compressed_image = np.zeros((new_height, new_width), dtype=np.uint8)

    # Process image in 8x8 blocks
    for i in range(0, new_height, 8):
        for j in range(0, new_width, 8):
            block = image[i:i+8, j:j+8]

            # DCT
            dct_block = cv2.dct(np.float32(block) - 128) #to center values around zero

            # Quantize DCT coefficients
            quantized_block = np.round(dct_block / Q) * Q

            # Inverse DCT
            idct_block = cv2.idct(quantized_block) + 128
            idct_block = np.clip(idct_block, 0, 255).astype(np.uint8)

            # Store in compressed image
            compressed_image[i:i+8, j:j+8] = idct_block

    # Remove padding to restore original size
    compressed_image = compressed_image[:height, :width]

    # Save the compressed image
    cv2.imwrite(output_path, compressed_image)
    print(f"DCT-based lossy compression done: {output_path}")

def compress_image_lossless(input_path, output_path):
    image = Image.open(input_path)
    image.save(output_path, format='PNG', optimize=True)
    print(f"Lossless compression (PNG) done: {output_path}")


input_image = "/content/zoro.jpg"
output_jpeg = "compressed_lossy.jpg"
output_png = "compressed_lossless.png"

dct_compress_image(input_image, output_jpeg, quality=50)
compress_image_lossless(input_image, output_png)

from IPython.display import display

original = Image.open(input_image)
display(original)


compressed_lossy = Image.open(output_jpeg)
display(compressed_lossy)


compressed_lossless = Image.open(output_png)
display(compressed_lossless)

Output hidden; open in https://colab.research.google.com to view.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# Load CIFAR-10 dataset
(cifar_X_train, cifar_y_train), (cifar_X_test, cifar_y_test) = cifar10.load_data()
cifar_X_train, cifar_X_test = cifar_X_train / 255.0, cifar_X_test / 255.0  # Normalize

cifar_y_train = to_categorical(cifar_y_train, 10)
cifar_y_test = to_categorical(cifar_y_test, 10)

X_train_cifar, X_val_cifar, y_train_cifar, y_val_cifar = train_test_split(cifar_X_train, cifar_y_train, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train_cifar)

# CNN Model
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Reduce Learning Rate on Plateau
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)

# Train CNN Model
cifar_model = create_cnn((32, 32, 3), 10)
cifar_model.fit(datagen.flow(X_train_cifar, y_train_cifar, batch_size=64),
                epochs=10,
                validation_data=(X_val_cifar, y_val_cifar),
                callbacks=[lr_reduction])

# Model Evaluation
def evaluate_model_with_roc_auc(model, X_test, y_test, num_classes):
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test_true = np.argmax(y_test, axis=1)

    print(classification_report(y_test_true, y_pred))
    cm = confusion_matrix(y_test_true, y_pred)
    print("Confusion Matrix:\n", cm)

    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred_prob.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"], label="Micro-average ROC curve (AUC = {:.2f})".format(roc_auc["micro"]))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label="Class {} (AUC = {:.2f})".format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.show()

    return cm, roc_auc

# Evaluate the CNN model
evaluate_model_with_roc_auc(cifar_model, cifar_X_test, cifar_y_test, 10)

# Optional: Transfer Learning with ResNet50
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze pretrained layers

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train ResNet50 Model
resnet_model = create_resnet_model((32, 32, 3), 10)
resnet_model.fit(datagen.flow(X_train_cifar, y_train_cifar, batch_size=64),
                 epochs=30,
                 validation_data=(X_val_cifar, y_val_cifar),
                 callbacks=[lr_reduction])

# Evaluate ResNet50 Model
evaluate_model_with_roc_auc(resnet_model, cifar_X_test, cifar_y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 48s 54ms/step - accuracy: 0.2748 - loss: 2.1998 - val_accuracy: 0.3742 - val_loss: 1.8025 - learning_rate: 0.0100
Epoch 2/50
369/625 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.4007 - loss: 1.6634

KeyboardInterrupt: 